In [25]:
# Importing Modules

import time, math, random
import numpy as np
# from multipledispatch import dispatch

In [2]:
# Colours

normal = "\033[0;37;48m"
yellow = "\033[0;33;48m"

orange = "\033[0;34;40m"
green = "\033[0;32;40m"
blue = "\033[0;35;40m"

In [3]:
# Files for importing/exporting

file_ID_storage = "ID Storage.txt"
file_habits_info = "Habits Info.txt"
file_logs = "Logs.txt"

In [4]:
# Global Variables

weight_limit = 5

colour_classifier = "\033"
colour_length = len(normal)
time_colour = yellow

display_indentation = "   "
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

ID_length = 3

In [35]:
# Mathematical functions

def softmax(array):
    np_array = np.array(array)
    exponentiated_array = np.exp(np_array)
    normalized_array = exponentiated_array / np.sum(exponentiated_array)

    return list(normalized_array)

In [36]:
# Stylization functions

def remove_colour(message: str) -> str:
    uncoloured_message = []

    for word in message.split(" "):
        if colour_classifier in word:
            classifier_index = word.index(colour_classifier)
            word = word[:classifier_index] + word[classifier_index + colour_length:]

        uncoloured_message.append(word)

    return " ".join(uncoloured_message)

def get_stylized_time() -> str:
    current_time = time.strftime("%I:%M:%S %p  %d-%b-%Y")
    return f"{time_colour}{current_time}{normal}"

In [37]:
class Common_Functions:    
    def stylized_name(self) -> str:
        return f"{self.colour} {self.name_prefix} {self.name} {normal}"
    
    def log(self, message: str, display_message = True) -> None:
        log_message = f"({get_stylized_time()})  {self.stylized_name()}: {message}\n"

        with open(file_logs, "a", encoding = 'utf-8') as file:
            file.write(remove_colour(log_message))

        if display_message:
            print(log_message)

    def log_error(self, error_type, error_message: str) -> None:
        self.log(error_message, False)
        raise error_type(error_message)

In [38]:
class Habit(Common_Functions):
    def __init__(self, name: str, weight: float, repeat_days: list | tuple) -> None:
        self.name = name
        self.weight = weight
        self.category = None
        self.repeat_days = repeat_days
        
        self.name_prefix = "#"
        self.colour = blue

        self.set_name(name, False)
        self.set_weight(weight, False)
        self.set_repeat_days(repeat_days, False)

        self.log(f"Made {self.stylized_name()}")

    def set_name(self, name: str, record_log = True) -> None:
        raise_error = False
        if not (self.category is None):
            raise_error = self.category.habit_exists(name)

        if raise_error:
            self.log_error(ValueError, f"A habit with the name \"{name}\" already exists!")

        self.name = name

        if record_log:
            self.log(f"Modified {self.stylized_name()}'s name")

    def set_weight(self, weight: float, record_log = True) -> None:
        if weight > weight_limit:
            self.log_error(ValueError, f"Weights can't have a value greater than {weight_limit}!")
        
        self.weight = weight

        if record_log:
            self.log(f"Modified {self.stylized_name()}'s weight")

    def set_category(self, category):
        self.category = category

    def set_repeat_days(self, selected_days: list | tuple, record_log = True) -> None:
        selected_days = [days[index - 1] for index in selected_days]
        self.repeat_days = selected_days

        self.log(f"Modified the repeat days", record_log)

In [39]:
class Habit_List(Common_Functions):
    def __init__(self, name: str) -> None:
        self.name = name
        self.habits = []

        self.name_prefix = ">"
        self.colour = green

        self.log(f"Made {self.stylized_name()}")

    def net_weight(self) -> float:
        return sum(list(map(lambda habit: habit.weight, self.habits)))
    
    def display_habits(self) -> None:
        print(self.stylized_name())

        for index, habit in enumerate(self.habits):
            print(f"{display_indentation}{index + 1}. {habit.stylized_name()}")

    def habit_exists(self, name: str):
        exists = False
        if self.name == name:
                exists = True

        for habit in self.habits:
            if habit.name == name:
                exists = True
                break

        return exists

    def add_habit(self, habit: Habit) -> None:
        if self.habit_exists(habit.name):
            self.log_error(ValueError, f"A habit with the name \"{habit.name}\" already exists in this list!")

        self.habits.append(habit)
        habit.set_category(self)

        self.log(f"Added {habit.stylized_name()}")

    def delete_habit(self, name: str):
        required_habit = self.get_habit_from_name(name)
        self.habits.remove(required_habit)
        self.log(f"Deleted {required_habit.stylized_name()}")

    def get_habit_from_name(self, name: str):
        required_habit = None
        for habit in self.habits:
            if habit.name == name:
                required_habit = habit
                break

        if required_habit is None:
            self.log_error(ValueError, f"No habit with the name \"{name}\" exists in this list!")
        
        self.log(f"Extracted {required_habit.stylized_name()}", False)
        return required_habit

    def swap(self, habit_1_index: int, habit_2_index: int) -> None:
        self.habits[habit_1_index], self.habits[habit_2_index] = self.habits[habit_2_index], self.habits[habit_1_index]
        self.log(f"Swaped the order of {self.habits[habit_2_index].stylized_name()} and {self.habits[habit_1_index].stylized_name()}")

    def change_position(self, habit_index: int, to_index: int) -> None:
        habit = self.habits.pop(habit_index - 1)
        self.habits.insert(to_index - 1, habit)

        self.log(f"Shifted {habit.stylized_name()} to Position {to_index}")

    def export_habits(self) -> None:
        with open(file_habits_info, "w", encoding = 'utf-8') as file:
            contents = []
            for habit in self.habits:
                export_format = list(map(str, [habit.category.name, habit.name, habit.weight]))
                contents.append(",".join(export_format) + "\n")

            file.writelines(contents)

In [41]:
playing = Habit_List("Playing 🏃‍♂️")

habits = [Habit("Table Tennis 🏓", 2, [1,3,5,7]),
          Habit("Basketball 🏀", 3, [2,6]),
          Habit("Chess ♟️", 3.5, [1,2,3,4,5]),
          Habit("Badminton 🏸", 2.5, [4,6,7])]

for habit in habits:
    playing.add_habit(habit)

(11:01:09 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Made  > Playing 🏃‍♂️ 

(11:01:09 PM  22-Jul-2024)   # Table Tennis 🏓 : Made  # Table Tennis 🏓 

(11:01:09 PM  22-Jul-2024)   # Basketball 🏀 : Made  # Basketball 🏀 

(11:01:09 PM  22-Jul-2024)   # Chess ♟️ : Made  # Chess ♟️ 

(11:01:09 PM  22-Jul-2024)   # Badminton 🏸 : Made  # Badminton 🏸 

(11:01:09 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Added  # Table Tennis 🏓 

(11:01:09 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Added  # Basketball 🏀 

(11:01:09 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Added  # Chess ♟️ 

(11:01:09 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Added  # Badminton 🏸 



In [42]:
playing.display_habits()

 > Playing 🏃‍♂️ 
   1.  # Table Tennis 🏓 
   2.  # Basketball 🏀 
   3.  # Chess ♟️ 
   4.  # Badminton 🏸 


In [43]:
playing.export_habits()

In [68]:
class Habit_Tracker(Common_Functions):
    def __init__(self, category: Habit_List) -> None:
        self.name = "Habit Tracker 📅"
        self.habits_today = {}
        self.category = category

        self.name_prefix = "~"
        self.colour = orange

        self.log(f"Made {self.stylized_name()}")

    def get_habit_from_name(self, name: str):
        required_habit = None
        for habit in list(self.habits_today.keys()):
            if habit.name == name:
                required_habit = habit
                break

        if required_habit is None:
            self.log_error(ValueError, f"No habit with the name \"{name}\" is present in today's habits!")
        
        self.log(f"Extracted {required_habit.stylized_name()}", False)
        return required_habit

    def generate_today(self) -> None:
        day = time.strftime("%a")
        habits_today = {}

        for habit in self.category.habits:
            if day in habit.repeat_days:
                habits_today.update({habit : 0})

        self.habits_today = habits_today
        self.log(f"Generated today's habits")

    def completed_habit(self, name: str) -> None:
        habit = self.get_habit_from_name(name)
        self.habits_today[habit] = 1

        self.log(f"Completed {habit.stylized_name()}")
        
    
    def progress(self) -> float:
        habit_weights = list(map(lambda habit: habit.weight, self.habits_today.keys()))
        habit_status = np.array(list(self.habits_today.values()))

        softmaxed_weights = np.array(softmax(habit_weights))
        
        return round((np.sum(softmaxed_weights[habit_status == 1]) / np.sum(softmaxed_weights)) * 100, 2)
    
    def display_habits_today(self) -> None:
        print(f"{self.stylized_name()} Today {self.name_prefix}")
        for index, habit in enumerate(list(self.habits_today.keys())):
            print(f"{display_indentation}{index + 1}. {habit.stylized_name()} - {habit.weight} - {self.habits_today[habit]}")

        print(f"\n{time_colour}Progress:{normal} {self.progress()}")

In [69]:
habit_track = Habit_Tracker(playing)
habit_track.generate_today()

(11:08:17 PM  22-Jul-2024)   ~ Habit Tracker 📅 : Made  ~ Habit Tracker 📅 

(11:08:17 PM  22-Jul-2024)   ~ Habit Tracker 📅 : Generated today's habits



In [70]:
habit_track.completed_habit("Chess ♟️")

(11:08:17 PM  22-Jul-2024)   ~ Habit Tracker 📅 : Completed  # Chess ♟️ 



In [71]:
habit_track.display_habits_today()

 ~ Habit Tracker 📅  Today ~
   1.  # Table Tennis 🏓  - 2 - 0
   2.  # Chess ♟️  - 3.5 - 1

Progress: 81.76


In [72]:
habit_track.progress()

81.76

In [24]:
# Goal - Track dates with respect to their habits

def generate_random_ID() -> int:
    with open(file_ID_storage, "r") as file:
        extracted_IDs = file.readlines()
        used_IDs = list(map(lambda id: int(id.removesuffix("\n")), extracted_IDs))

    while True:
        generated_ID = round((random.random() * (10 ** ID_length)), 0)
        if not (generated_ID in used_IDs):
            break

    return int(generated_ID)

generate_random_ID()

6785